## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
from pyspark.sql.functions import current_timestamp, lit, input_file_name
from pyspark.sql.types import _parse_datatype_string
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import ArrayType, StringType
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import regexp_extract, split, explode, col,to_timestamp, regexp_replace
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql.functions import isnan, when, count, isnull
import matplotlib.pyplot as plt
import pyspark.pandas as ps
import mlflow.pyfunc
from pyspark.sql.functions import udf
import seaborn as sns
from transformers import pipeline
from sklearn.metrics import confusion_matrix
from delta import DeltaTable
spark = SparkSession.builder. appName("MyApp").config("spark.sql.shuffle.partitions", 12).getOrCreate()

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
#Read, Count, and Print contents of file
source_df = get_source_listing_df()
print(source_df)
print(len(source_df))
show_s3_file_contents("voc_volume/1000.json")

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
#Define Schema and Readstream 
raw_schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True)
])

raw_tweets_df = (
    spark.readStream.format("cloudfiles").schema(raw_schema).option('cloudfiles.format', 'json').load(TWEET_SOURCE_PATH)
)

In [0]:
#Check Schema 

assert raw_tweets_df.schema == raw_schema, "File not present in Raw Path"
print("Assertion passed.")

In [0]:
#Merging new Schema to Table
additional_schema = [
    StructField("source_file", StringType(), False), StructField("processing_time", TimestampType(), False)
]
bronze_schema = StructType(list(raw_schema) + additional_schema)

In [0]:
#Transform Raw data to Bronze data according to data dictionary
raw_tweets_df = raw_tweets_df.select(
    "date", "user","text","sentiment", 
    input_file_name().alias("source_file"),
    current_timestamp().alias("processing_time"),
)

In [0]:
#Check Schema
assert raw_tweets_df.schema == bronze_schema, "File not present in Raw Path"
print("Assertion passed.")

In [0]:
#Write Stream, using the Bronze Checkpoint and Bronze Delta, named bronze_stream, takes about 45 minutes to run
query_bronze = raw_tweets_df.writeStream.format("delta").outputMode("append").option("checkpointLocation", BRONZE_CHECKPOINT).partitionBy("processing_time").queryName("bronze_stream").start(BRONZE_DELTA)

In [0]:
#Stop Stream when input and processed rows per second is 0
stream_started = False
last_progress = False
watch_query = False
while True:
    time.sleep(30)
    if not stream_started:
        stream_started = wait_stream_start(spark, "bronze_stream")
    if stream_started and not last_progress:
        if(query_bronze.lastProgress is None):
            last_progress = False
        if(query_bronze.lastProgress is not None):
            last_progress = True

    if stream_started and last_progress and not watch_query:
        watch_query = watch(query_bronze)

    if stream_started and last_progress and watch_query:
        query_bronze.stop()
        query_bronze.awaitTermination() 
        break  
    time.sleep(30)


In [0]:
#Loading Delta Table into DF and viewing it
bronze_tweets = spark.read.format("delta").load(BRONZE_DELTA)
display(bronze_tweets)


## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
#Number of Tweets Captured by Bronze Table
print(bronze_tweets.count())

In [0]:
#Columns that contain NAN or Null Values ~ No nan or null values
bronze_tweets.select([count(when(isnull(c), c)).alias(c) for c in bronze_tweets.columns]).show()

In [0]:
#Tweets by unique user handle and sort in descending count 
counts = bronze_tweets.groupBy("user").count().orderBy(col('count').desc())
counts.show()

In [0]:

bronze_tweets_contains = bronze_tweets.filter(F.col("text").contains("@"))
#Tweets that have at least one mention
print(bronze_tweets_contains.count())
#Tweets that have no mentions
print(bronze_tweets.count() - bronze_tweets_contains.count())


In [0]:
#Bar Plot of top 20 tweets
counts = counts.limit(20)
counts = ps.DataFrame(counts)
counts.plot.bar(x = "user", y = "count", title = "Top 20 Tweeters")

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
#Setup Readstream on Bronze Table
bronze_df = spark.readStream.format("delta").option("basePath", BRONZE_DELTA).load(BRONZE_DELTA)

In [0]:
#Transform Bronze Data to Silver Data 
silver_schema = "timestamp TIMESTAMP, mention STRING, cleaned_text STRING, Sentiment STRING"
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
filtered_bronze_df = bronze_df.filter(bronze_df["text"].like("%@%")) #Filter Dataframe to only get tweets with mentions
filtered_bronze_df = filtered_bronze_df.withColumn("mid", F.split('text'," ")) #Split each word in text 

def filter_array(arr): #Filter so that only words with @ are left
    return [x for x in arr if "@" in x]

filter_array_udf = udf(filter_array, ArrayType(StringType())) 
filtered_bronze_df = filtered_bronze_df.withColumn("mention", filter_array_udf(col("mid"))) 
silver_tweets = filtered_bronze_df.select(to_timestamp('date', format='EEE MMM dd HH:mm:ss zzz yyyy').alias("timestamp"), "mention", regexp_replace("text", r'@\w+', "").alias("cleaned_text"), "sentiment")
silver_tweets = silver_tweets.withColumn("mention", F.explode("mention")) #explode so that each mention gets its own row
silver_tweets = silver_tweets.where(silver_tweets.mention != '@') #delete rows where there is just @ and no mention

In [0]:
#Set up WriteStream, using Silver Checkpoint and Silver Delta
query_silver = silver_tweets.writeStream.format("delta").outputMode("append").option("checkpointLocation", SILVER_CHECKPOINT).queryName("silver_stream").start(SILVER_DELTA)

In [0]:
#Logic to Stop Stream when complete
stream_started = False
last_progress = False
query_triggered = False
while True:
    time.sleep(30)
    if not stream_started:
        stream_started = wait_stream_start(spark, "silver_stream")
    if stream_started and not last_progress:
        if(query_silver.lastProgress is None):
            last_progress = False
        if(query_silver.lastProgress is not None):
            last_progress = True
    if stream_started and last_progress and not query_triggered:
        query_triggered = watch(query_silver)
    if stream_started and last_progress and query_triggered:
        query_silver.stop()
        query_silver.awaitTermination()  
        break 
    time.sleep(30)


In [0]:
#Load table and display to check 
silver_df = spark.read.format('delta').load(SILVER_DELTA)
display(silver_df)

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
#Set up Read Stream from Silver Table
silver_table= spark.readStream.format("delta").option("basePath", SILVER_DELTA).load(SILVER_DELTA)

In [0]:
#Load Pretrained Classifier from MLFlowRegistry 
analyze_sentiment_udf = mlflow.pyfunc.spark_udf(spark, "models:/" + MODEL_NAME + "/1")
silver_table = silver_table.withColumn("mid", analyze_sentiment_udf(col("cleaned_text")))



In [0]:
#Separating results from Model into Predicted Score and Predicted Sentiment
from pyspark.sql.functions import split
silver_table = silver_table.select('timestamp', 'mention', 'cleaned_text', 'sentiment',
                silver_table["mid"].score.alias('predicted_score'), 
                silver_table["mid"].label.alias('predicted_sentiment')) 


In [0]:
#Converting Sentiment to sentiment_id
silver_table = silver_table.withColumn('sentiment_id', F.when(F.col("sentiment")== "positive", 1).otherwise(F.when(F.col("sentiment")== "negative", 0)))

#Setting the Neutral tweets into Negative because original sentiment_id is just positive or negative
silver_table = silver_table.withColumn('predicted_sentiment', regexp_replace('predicted_sentiment', 'NEU', 'NEG'))

#Convert Predicted Sentiment into Sentiment ID, and Also converting Predicted Score to scale from 0 to 100, per the data dictionary
silver_table = silver_table.withColumn('predicted_sentiment_id', F.when(F.col("predicted_sentiment")== "POS", 1).otherwise(F.when(F.col("predicted_sentiment")== "NEG", 0))).withColumn("predicted_score", (col("predicted_score").cast("double") * 100))



In [0]:
 #WriteStream to Gold table, using GOLD_CHECKPOINT and GOLD_DELTA, and named gold_stream, takes about 1.5 hours to run
query_gold = silver_table.writeStream.format("delta").outputMode("append").option("checkpointLocation",GOLD_CHECKPOINT).queryName("gold_stream").start(GOLD_DELTA)


In [0]:
#Logic to Stop Stream when complete
stream_started = False
last_progress= False
watch_query = False
while True:
    time.sleep(30)
    if not stream_started:
        stream_started = wait_stream_start(spark, "gold_stream")
    if stream_started and not last_progress:
        if(query_gold.lastProgress is None):
            last_progress = False
        if(query_gold.lastProgress is not None):
            last_progress = True
    if stream_started and last_progress and not watch_query:
        watch_query = watch(query_gold)

    if stream_started and last_progress and watch_query:
        query_gold.stop()
        query_gold.awaitTermination()   
        break
    time.sleep(30)


In [0]:
#Read in DF and Display to Check
gold_table = spark.read.format('delta').load(GOLD_DELTA)
display(gold_table)

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
with mlflow.start_run():
    #Get the Multiclass Metrics
    prediction_and_label = gold_table.select(col("predicted_sentiment_id").cast("double"), col("sentiment_id").cast("double"))
    multi_metrics = MulticlassMetrics(prediction_and_label.rdd)

    #Build confusion Matrix
    conf_matrix = multi_metrics.confusionMatrix()
    conf_matrix_array = conf_matrix.toArray()
    plt.imshow(conf_matrix_array, cmap='Blues', interpolation='nearest')
    for i in range(len(conf_matrix_array)):
        for j in range(len(conf_matrix_array[i])):
            plt.text(j, i, format(conf_matrix_array[i, j], '.2f'),
                    horizontalalignment="center",
                    color="black")

    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.colorbar()
    plt.xticks(range(conf_matrix_array.shape[1]))
    plt.yticks(range(conf_matrix_array.shape[0]))
    plt.savefig("confusion_matrix.png")

    #Log the Parameters and Metrics
    mlflow.log_artifact("confusion_matrix.png")
    mlflow.log_param("model_name", MODEL_NAME)
    mlflow.log_param("mlflow_version", 1)
    deltaTable = DeltaTable.forPath(spark, SILVER_DELTA)
    history_df = deltaTable.history().select("version").orderBy("version", ascending=False)
    version = history_df.collect()[0][0]
    mlflow.log_param("delta_table_version", version)
    mlflow.log_metric('precision', multi_metrics.weightedPrecision)
    mlflow.log_metric('recall', multi_metrics.weightedRecall)
    mlflow.log_metric('FScore', multi_metrics.weightedFMeasure())

    mlflow.end_run()



## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
#Total Mentions in Gold Data
print(gold_table.count())

#Total Distinct Mentions in Gold Data
print(gold_table.select("mention").distinct().count())

In [0]:
from pyspark.sql.functions import col, min,max
#Find Oldest and Newest Timestamp for Each Mention
application_data = gold_table.groupBy("mention").agg(min(col("timestamp")).alias("oldest_timestamp"), max(col("timestamp")).alias("newest_timestamp")
)

In [0]:
#Capture Number of Positive and Negative tweets for each Mention (No Neutral since already included them in Negative)
aggregate = gold_table.groupBy("mention", "predicted_sentiment").count().groupBy("mention").pivot("predicted_sentiment").agg({"count": "sum"}).fillna(0)
#Capture Total Number of Tweets for Each Mention
application_data = application_data.join(aggregate, "mention", "left")
total_count = gold_table.groupBy("mention").count().withColumnRenamed("count","total")
application_data = application_data.join(total_count, "mention", "left")
#Order Total Tweets by Descending Order
application_data = application_data.orderBy(col('total').desc())
application_data = application_data.withColumnRenamed("POS", "positive")
application_data = application_data.withColumnRenamed("NEG", "negative")
display(application_data)

In [0]:
#Bar Graph of Top 20 Mentions with Positive Sentiment
positive = application_data.orderBy(col('positive').desc())
positive = positive.limit(20)
positive = ps.DataFrame(positive)
positive.plot.bar(x = "mention", y = "positive", title = "Top 20 Mentions with Positive Sentiment")

In [0]:
#Bar Graph of Top 20 Mentions with Negative Sentiment
negative = application_data.orderBy(col('negative').desc())
negative = negative.limit(20)
negative = ps.DataFrame(negative)
negative.plot.bar(x = "mention", y = "negative", title = "Top 20 Mentions with Negative Sentiment")


## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
#print and stop streams
for stream in spark.streams.active:
    print(stream.name)
stop_all_streams()


In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()
elapsed_time = END_TIME - START_TIME
print(elapsed_time)

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE